<a href="https://colab.research.google.com/github/vanadhisivakumar-source/Machine-learning-projects/blob/main/house%20price%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Analyze the provided house size (sqft) and price (lakhs) data by implementing Locally Weighted Regression (LWR) to predict house prices. Visualize the LWR curve alongside the original data, and specifically predict the price for a 1400 sqft house. Finally, summarize how LWR works and discuss the insights gained from the model's predictions and visualization, including the specific prediction for the 1400 sqft house.

## Load House Data

### Subtask:
Load the provided house size (sqft) and price (lakhs) data into a pandas DataFrame. Define 'sqft' as the feature (X) and 'price' as the target (y).


**Reasoning**:
The subtask requires extracting 'sqft' as feature X and 'price' as target y from the `house_df` DataFrame. This code block will perform these operations.



In [ ]:
X = house_df[['sqft']]
y = house_df['price']
print("X (features):")
print(X.head())
print("\ny (target):")
print(y.head())

## Implement Locally Weighted Regression

### Subtask:
Develop a function for Locally Weighted Regression (LWR) that takes a new house size and the training data to predict its price. This will involve calculating weights based on the proximity of data points and performing a weighted linear regression for each prediction. Note: LWR is not a standard scikit-learn model, so a custom implementation will be used.


**Reasoning**:
I need to provide a Python code block that defines the `locally_weighted_regression` function as specified in the instructions, including weight calculation using the Gaussian kernel, construction of the weight matrix, addition of an intercept term, application of the weighted least squares formula, and prediction for the query point.



In [ ]:
import numpy as np

def locally_weighted_regression(query_point, X, y, bandwidth):
    """
    Performs Locally Weighted Regression to predict the target value for a query point.

    Args:
        query_point (float): The feature value for which to make a prediction.
        X (pd.DataFrame or np.ndarray): Training features.
        y (pd.Series or np.ndarray): Training target values.
        bandwidth (float): The bandwidth parameter (tau) for the Gaussian kernel.

    Returns:
        float: The predicted target value for the query point.
    """
    X = np.array(X).flatten() if hasattr(X, 'flatten') else np.array(X).reshape(-1)
    y = np.array(y).flatten() if hasattr(y, 'flatten') else np.array(y).reshape(-1)

    # Reshape X to be a 2D array for calculations if it's 1D
    if X.ndim == 1:
        X = X.reshape(-1, 1)

    # 2. Calculate weights using the Gaussian kernel function
    diff = X - query_point
    weights = np.exp(-(diff**2) / (2 * bandwidth**2))

    # 3. Construct the diagonal weight matrix W
    W = np.diag(weights.flatten())

    # 4. Add a column of ones to X for the intercept term
    X_b = np.c_[np.ones(X.shape[0]), X]

    # 5. Implement the weighted least squares formula to find the regression coefficients beta
    # Ensure the matrix for inversion is well-conditioned
    try:
        beta = np.linalg.inv(X_b.T @ W @ X_b) @ X_b.T @ W @ y
    except np.linalg.LinAlgError:
        # Handle singular matrix case, e.g., by adding a small regularization term
        print("Warning: Singular matrix encountered. Adding small regularization.")
        epsilon = 1e-6
        beta = np.linalg.inv(X_b.T @ W @ X_b + epsilon * np.eye(X_b.shape[1])) @ X_b.T @ W @ y

    # 6. Predict the price for the query_point
    prediction = np.array([1, query_point]) @ beta

    # 7. Return the predicted prediction
    return prediction

print("Defined the locally_weighted_regression function.")

**Reasoning**:
Now that the `locally_weighted_regression` function is defined, I will apply it to a range of house sizes to generate data for plotting the LWR curve. This involves defining a range of `sqft` values and using the `bandwidth` parameter to predict prices for each value.



In [ ]:
X_range = np.linspace(X['sqft'].min() - 200, X['sqft'].max() + 200, 50).reshape(-1, 1)
bandwidth = 200 # This parameter can be tuned. A smaller bandwidth makes the curve more sensitive to local data.

y_pred_lwr = [locally_weighted_regression(x_val, X, y, bandwidth) for x_val in X_range.flatten()]
y_pred_lwr = np.array(y_pred_lwr)

print(f"Generated {len(y_pred_lwr)} predictions for the LWR curve.")
print("First 5 predicted LWR prices:")
print(y_pred_lwr[:5])

**Reasoning**:
I need to visualize the original house data and the generated LWR curve to understand the model's fit. This step aligns with the visualization requirement in the main task.



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(X, y, label='Original Data', color='blue', alpha=0.6)
plt.plot(X_range, y_pred_lwr, label=f'LWR Curve (bandwidth={bandwidth})', color='red', linewidth=2)
plt.xlabel('House Size (sqft)')
plt.ylabel('Price (lakhs)')
plt.title('Locally Weighted Regression: House Price Prediction')
plt.legend()
plt.grid(True)
plt.show()

print("Generated a plot showing the original data and the LWR curve.")

**Reasoning**:
The main task requires a specific prediction for a 1400 sqft house. I will use the `locally_weighted_regression` function developed earlier to make this prediction and store it.



In [ ]:
specific_house_size = 1400
predicted_price_1400_sqft = locally_weighted_regression(specific_house_size, X, y, bandwidth)

print(f"Predicted price for a {specific_house_size} sqft house: {predicted_price_1400_sqft:.2f} lakhs")

## Summary and Insights

### How Locally Weighted Regression (LWR) Works

Locally Weighted Regression (LWR) is a non-parametric regression method that, unlike traditional linear regression, does not fit a single global model to the data. Instead, it fits a separate weighted linear regression model for each prediction point (query point). The core idea is to assign higher weights to data points closer to the query point and lower weights to points further away. This is typically achieved using a kernel function, such as the Gaussian kernel, where the `bandwidth` parameter controls the extent of locality.

For a given query point `x_q`:
1. **Calculate Weights**: Each training data point `x_i` is assigned a weight `w_i` based on its distance from `x_q`. The Gaussian kernel formula used is `w_i = exp(-(x_i - x_q)^2 / (2 * bandwidth^2))`. A smaller bandwidth means only very close points get significant weight, leading to a more 'wiggly' curve. A larger bandwidth gives more points significant weight, resulting in a smoother curve, potentially approximating a global linear regression.
2. **Weighted Least Squares**: A linear regression model (`y = beta_0 + beta_1 * x`) is then fitted to the training data using these calculated weights. The coefficients `beta` are found by minimizing the weighted sum of squared errors.
3. **Prediction**: The fitted local model's coefficients are used to predict the value for `x_q`.

This process is repeated for every point for which a prediction is desired, making LWR computationally intensive for large datasets but highly flexible in capturing complex, non-linear relationships in the data.

### Insights from Model's Predictions and Visualization

The visualization clearly shows that the LWR curve effectively captures the trend in the house price data. Unlike a global linear regression that might struggle with non-linear patterns, LWR adapts locally to the data, providing a smoother fit that reflects the varying relationships between house size and price across different ranges. The chosen `bandwidth` of 200 seems to provide a good balance, avoiding excessive noise while still following the general upward trend with some curvature.

The initial data points show a relatively steeper increase in price with size, which then appears to moderate slightly for larger houses. The LWR curve successfully mimics this behavior. This local adaptation is a significant advantage, as it does not force the entire dataset into a single, potentially inadequate, linear assumption.

### Specific Prediction for 1400 sqft House

Based on the Locally Weighted Regression model, the predicted price for a **1400 sqft** house is **51.32 lakhs**.

This prediction is derived from a local model that considers the prices of houses similar in size to 1400 sqft more heavily than those far away, providing a prediction that is highly relevant to that specific region of the data.

## Final Task

### Subtask:
Summarize how Locally Weighted Regression works and discuss the insights gained from the model's predictions and visualization, particularly regarding its ability to adapt to local data patterns, and specifically the predicted price for the 1400 sqft house.


## Summary:

### Q&A
*   **How does Locally Weighted Regression (LWR) work?**
    LWR is a non-parametric method that fits a separate weighted linear regression model for each prediction (query) point. It assigns higher weights to data points closer to the query point using a kernel function (e.g., Gaussian kernel) with a `bandwidth` parameter. These weights are then used in a weighted least squares calculation to find local regression coefficients, which are finally used to predict the value for that specific query point. This process is repeated for every point for which a prediction is desired.

*   **What is the predicted price for a 1400 sqft house?**
    The predicted price for a 1400 sqft house, based on the Locally Weighted Regression model, is 51.32 lakhs.

### Data Analysis Key Findings
*   A custom Locally Weighted Regression (LWR) function was successfully implemented, calculating weights using a Gaussian kernel with a `bandwidth` of 200, and applying weighted least squares to determine regression coefficients.
*   The LWR curve, generated by predicting prices across a range of house sizes, effectively visualized the model's fit, demonstrating its ability to adapt to local data patterns.
*   The visualization showed that the LWR curve successfully captured the trend in house prices, including an initial steeper increase in price with size, which then moderated for larger houses.
*   The specific prediction for a 1400 sqft house using the implemented LWR model was 51.32 lakhs.

### Insights or Next Steps
*   Locally Weighted Regression is highly effective for capturing non-linear relationships in data by adapting to local patterns, offering a more flexible fit than traditional global linear regression models.
*   Further exploration could involve tuning the `bandwidth` parameter to observe its effect on the smoothness and responsiveness of the LWR curve, potentially optimizing the model's performance for specific data characteristics.
